In [ ]:
!pip install tflearn
!pip install tensorflow-gpu

In [2]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout,Add,SeparableConv2D,ReLU




from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import math
import random


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
device_list=tf.test.gpu_device_name()
device_list
if(device_list!='/device:GPU:0'):
  raise SystemError('GPU')
  print(format(device_list))

In [4]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import opendatasets as dt
dt.download("https://www.kaggle.com/competitions/nn23-sports-image-classification/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: yousefalaa86
Your Kaggle Key: ··········


100%|██████████| 262M/262M [00:01<00:00, 154MB/s]



Extracting archive ./nn23-sports-image-classification/nn23-sports-image-classification.zip to ./nn23-sports-image-classification


In [6]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten,Dropout
from keras.preprocessing.image import ImageDataGenerator

In [7]:
import os
os.mkdir('/content/images')

In [8]:
os.mkdir('/content/images/Basketball')
os.mkdir('/content/images/Football')
os.mkdir('/content/images/Rowing')
os.mkdir('/content/images/Swimming')
os.mkdir('/content/images/Tennis')
os.mkdir('/content/images/Yoga')

In [9]:
TRAIN_DIR = 'nn23-sports-image-classification/Train'
import shutil
for img in tqdm(os.listdir(TRAIN_DIR)):
  word_label = img.split('_')[0]
  if word_label == 'Basketball':
      shutil.move('/content/nn23-sports-image-classification/Train/'+img,'/content/images/Basketball/'+img)
  elif word_label == 'Football':
      shutil.move('/content/nn23-sports-image-classification/Train/'+img,'/content/images/Football/'+img)
  elif word_label == 'Rowing':
      shutil.move('/content/nn23-sports-image-classification/Train/'+img,'/content/images/Rowing/'+img)
  elif word_label == 'Swimming':
      shutil.move('/content/nn23-sports-image-classification/Train/'+img,'/content/images/Swimming/'+img)
  elif word_label == 'Tennis':
      shutil.move('/content/nn23-sports-image-classification/Train/'+img,'/content/images/Tennis/'+img)
  elif word_label == 'Yoga':
      shutil.move('/content/nn23-sports-image-classification/Train/'+img,'/content/images/Yoga/'+img)

100%|██████████| 1681/1681 [00:00<00:00, 31876.02it/s]


In [14]:
data='/content/images/'
mostafa,validation=tf.keras.utils.image_dataset_from_directory(
    data,
    validation_split=0.2,
    subset='both',
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=16,
    image_size=(299, 299),
    shuffle=True,
    seed=10,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 1681 files belonging to 6 classes.
Using 1345 files for training.
Using 336 files for validation.


In [15]:
#Alex net model
Alex_model = keras.models.Sequential([
    #data_augmention,
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(299,299,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    ########################
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(6, activation='softmax')
])


Alex_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 73, 73, 96)        34944     
                                                                 
 batch_normalization_5 (Batc  (None, 73, 73, 96)       384       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 36, 36, 96)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 36, 36, 256)       614656    
                                                                 
 batch_normalization_6 (Batc  (None, 36, 36, 256)      1024      
 hNormalization)                                                 
                                                      

In [16]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/best_weights_model_2',
    verbose=1,
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
opt = keras.optimizers.Adam(lr=0.0001)

Alex_model.compile(loss='sparse_categorical_crossentropy' ,optimizer=opt, metrics=['accuracy'])



Alex_model.fit(mostafa,epochs=100,validation_data=validation,callbacks=[model_checkpoint_callback])
##############################################################


Train on 85 steps, validate on 21 steps
Epoch 1/100
85/85 [==============================] - ETA: 0s - batch: 42.0000 - size: 1.0000 - loss: 4.6211 - acc: 0.4595

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2333: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates



Epoch 1: val_acc improved from -inf to 0.22619, saving model to /content/best_weights_model_2
85/85 [==============================] - 15s 151ms/step - batch: 42.0000 - size: 1.0000 - loss: 4.6211 - acc: 0.4595 - val_loss: 13.3563 - val_acc: 0.2262
Epoch 2/100
84/85 [============================>.] - ETA: 0s - batch: 41.5000 - size: 1.0000 - loss: 3.2091 - acc: 0.5774
Epoch 2: val_acc improved from 0.22619 to 0.41369, saving model to /content/best_weights_model_2
85/85 [==============================] - 12s 135ms/step - batch: 42.0000 - size: 1.0000 - loss: 3.3547 - acc: 0.5770 - val_loss: 3.9569 - val_acc: 0.4137
Epoch 3/100
84/85 [============================>.] - ETA: 0s - batch: 41.5000 - size: 1.0000 - loss: 2.7629 - acc: 0.6094
Epoch 3: val_acc improved from 0.41369 to 0.66667, saving model to /content/best_weights_model_2
85/85 [==============================] - 13s 150ms/step - batch: 42.0000 - size: 1.0000 - loss: 2.7850 - acc: 0.6089 - val_loss: 1.9360 - val_acc: 0.6667
Epoc

In [19]:
all_data=tf.keras.utils.image_dataset_from_directory(
    data,
    validation_split=None,
    subset=None,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(299, 299),
    shuffle=True,
    seed=10,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 1681 files belonging to 6 classes.


In [20]:
Alex_model.fit(all_data,epochs=35)

Train on 53 steps
Epoch 1/35
53/53 [==============================] - 8s 139ms/step - batch: 26.0000 - size: 1.0000 - loss: 1.2821 - acc: 0.9250
Epoch 2/35
53/53 [==============================] - 7s 117ms/step - batch: 26.0000 - size: 1.0000 - loss: 0.5093 - acc: 0.9429
Epoch 3/35
53/53 [==============================] - 7s 117ms/step - batch: 26.0000 - size: 1.0000 - loss: 0.1227 - acc: 0.9726
Epoch 4/35
53/53 [==============================] - 7s 116ms/step - batch: 26.0000 - size: 1.0000 - loss: 0.0613 - acc: 0.9869
Epoch 5/35
53/53 [==============================] - 7s 116ms/step - batch: 26.0000 - size: 1.0000 - loss: 0.0408 - acc: 0.9911
Epoch 6/35
53/53 [==============================] - 7s 117ms/step - batch: 26.0000 - size: 1.0000 - loss: 0.0265 - acc: 0.9935
Epoch 7/35
53/53 [==============================] - 8s 131ms/step - batch: 26.0000 - size: 1.0000 - loss: 0.0304 - acc: 0.9941
Epoch 8/35
53/53 [==============================] - 7s 115ms/step - batch: 26.0000 - size: 1.

In [21]:
TEST_DIR = '/content/nn23-sports-image-classification/Test'

def load_images_test_from_folder(folder):
    images = []
    images_names = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #img_normalized = img / 255.0
        img_data = cv2.resize(img, (299,299))
        if img is not None:
            images.append([np.array(img_data)])
            images_names.append(filename)
    return images , images_names



test,images_names = load_images_test_from_folder(TEST_DIR)
X_test = np.array([i for i in test]).reshape((-1, 299, 299, 3))

from tensorflow import keras
Alex_model = keras.models.load_model('/content/best_weights_model_2')

predictions = Alex_model.predict(X_test)


pred = []
for prediction in predictions:
    max_val = np.argmax(prediction)
    pred.append(max_val)

import csv
headers = ["image_name", "label"]
OutPut_list = []
for i in range(len(pred)):
    x = [images_names[i],pred[i]]
    OutPut_list.append(x)
with open("Sports.csv", "w") as Sport:
    student = csv.writer(Sport)
    student.writerow(headers)
    student.writerows (OutPut_list)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
